In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import warnings
warnings.filterwarnings('ignore')
import random as rnd
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
import matplotlib.patches as mpatches
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from scipy import stats
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
%matplotlib inline
SEED=4242
rnd.seed(10)
SIZE=30000
ALPHA=0.03
# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import collections
from sklearn.ensemble import RandomForestClassifier
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score,\
 accuracy_score, classification_report, make_scorer, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.utils.class_weight import compute_sample_weight
from imblearn.under_sampling import TomekLinks
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from xgboost import plot_importance
# Time Series Features
# import tsfresh as tsf
from datetime import date, timedelta
from scipy import signal
from scipy.stats import variation
# Imbalance handling
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTETomek

In [ ]:
df= pd.read_csv('/content/DFORE450.zip', compression='zip', low_memory=True)

In [ ]:
print(df.shape)
df.drop_duplicates(subset= 'email_address', keep='last', inplace= True)
print(df.shape)
df.set_index('email_address', inplace=True)

(79323, 669)
(76493, 669)


In [ ]:
bdex= list(filter(lambda x: '-' in x, df.columns.tolist()))
bdex.extend(['gender', 'birth_year'])
bf= df[bdex]
rf= df[df.columns.difference(bdex)]
print('BDEX Columns: ',bf.shape,'RFM Columns: ', rf.shape)

BDEX Columns:  (76493, 608) RFM Columns:  (76493, 60)


# RFM

In [ ]:
rf['Recency']= rf[['Vend_Recency', 'WalletLoad_Recency']].min(axis=1)
rf['DaysBetween']= rf[['AvgDaysBetweenVends', 'AvgDaysBetweenWalletLoad']].min(axis=1)
rf['DoB']= rf[['Vend_DaysonBook','WalletLoad_DaysonBook']].min(axis=1)
print(rf[['Recency', 'DaysBetween', 'DoB']].isnull().sum())
# Consumers in 150-180 cohort who visit atleast once in 10 days and their last activity is maximum 5 days old
flg1= (rf.Recency <= 5) & (rf.DaysBetween<=10) & (rf.DoB > 150) & (rf.DoB <= 180)
flg0= (rf.DoB > 150) & (rf.DoB <= 180)
print('Target Group: ', sum(flg1))
print('Size of Consumer Base: ', sum(flg0))
print('Minority Class Group: ', 100.00*sum(flg1)/sum(flg0))

Recency        0
DaysBetween    0
DoB            0
dtype: int64
Target Group:  1557
Size of Consumer Base:  6073
Minority Class Group:  25.638070146550305


In [ ]:
y=pd.Series(0*len(flg0), index=rf[flg0].index)
y[rf[flg1].index]=1

In [ ]:
bf1= bf.loc[rf[flg0].index]

In [ ]:
print( 'X values: ',bf1.shape, 'y values: ', y.shape)

X values:  (6073, 608) y values:  (6073,)


## Entropy calculation BF1

In [ ]:
prob= sum(y)/len(y)
entropy0= -(1-prob)*np.log2(1-prob) -prob*np.log2(prob)

In [ ]:
entropy_dict={}
for c in bf1.columns.tolist():
  prob_splt1= sum(y[bf1[bf1[c]==1].index])/len(y)
  entropy1= -(1-prob_splt1)*np.log2(1-prob_splt1) -prob_splt1*np.log2(prob_splt1)
  prob_splt2= sum(y[bf1[bf1[c]!=1].index])/len(y)
  entropy2= -(1-prob_splt2)*np.log2(1-prob_splt2) -prob_splt2*np.log2(prob_splt2)
  split_entrpy= (sum(bf1[c]==1)/ len(bf1[c]))*entropy1+ (sum(bf1[c]!=1)/ len(bf1[c]))*entropy2
  InfoGain= entropy0- split_entrpy
  print(c, round(prob_splt1, 3), round(entropy1, 3), round(prob_splt2,3), round(entropy2, 3), round(split_entrpy,3), round(InfoGain, 3))
  clear_output(wait= True)
  entropy_dict[c]= [round(prob_splt1, 3), round(entropy1, 3), round(prob_splt2,3), round(entropy2, 3), round(split_entrpy,3), round(InfoGain, 3)]

birth_year 0.0 nan 0.256 0.821 nan nan


In [ ]:
ef= pd.DataFrame.from_dict(entropy_dict, orient='index')

In [ ]:
ef.dropna(axis=0, inplace= True)

In [ ]:
ef.columns=['prob_splt1', 'entropy1', 'prob_splt2', 'entropy2', 'split_entrpy', 'InfoGain']

In [ ]:
ifcols= ef.loc[ef['InfoGain'] > .1].index.tolist()

In [ ]:
print( 'X values: ',bf1.shape, 'y values: ', y.shape)

X values:  (6073, 608) y values:  (6073,)


## Train/ Test

In [ ]:
train = bf1[ifcols].fillna(0)

In [ ]:
train.corrwith(y, axis=0).abs().sort_values(ascending= False)

COMPUTERS_INTERNET-WEB_BROWSING-VISIT-YELP_COM_LST_7D             0.041508
SHOPPING_CLOTHING-SHOPPED_ROSS_DRESS_FOR_LESS_LAST_3M             0.032209
COMPUTERS_INTERNET-VISITED_WEBSITE-DROPBOX_SITES_LAST_3_MONTHS    0.031468
SHOPPING_CLOTHING-BOUGHT_SNEAKER_ATHLETIC_SHOE_LAST_12M_REEBOK    0.030797
SHOPPING_HOMEANDGARDEN-LAUNDRY_DETERGENT-TIDE_HE                  0.028714
                                                                    ...   
COMPUTERS_INTERNET-VISIT-FACEBOOK_COM_IN_THE_LAST_30_DAYS         0.000852
COMPUTERS_INTERNET-VISIT-PANDORA_COM_IN_THE_LAST_30_DAYS          0.000566
PERSONA_EMPLOYMENT-SELF-EMPLOYED_WORK_OUTSIDE_OF_HOME             0.000233
COMPUTERS_INTERNET-VISITED_WEBSITE-NEUVOO_INC__LAST_3_MONTHS      0.000221
COMPUTERS_INTERNET-VISIT-INSTAGRAM_COM_IN_THE_LAST_7_DAYS         0.000082
Length: 94, dtype: float64

In [ ]:
train.columns[train.isnull().sum()> 0]

Index([], dtype='object')

In [ ]:
# Our data is already scaled we should split our training and test sets
from sklearn.model_selection import train_test_split

# This is explicitly used for undersampling.
X_train, X_test, y_train, y_test = train_test_split(train.values, y.values, test_size=0.3, random_state=4242, shuffle=True)

print('Training Sample- Minority Class: ', sum(y_train))
print('Total Training Sample: ', len(y_train))
print('%: ', 100*sum(y_train)/len(y_train))

Training Sample- Minority Class:  1080
Total Training Sample:  4251
%:  25.40578687367678


In [ ]:
# MinMax= MinMaxScaler()
# X_train= MinMax.fit_transform(X_train)
# y_train= y_train
# X_test= MinMax.transform(X_test)
# y_test= y_test

In [ ]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
%%time
from pandas.core.apply import ResamplerWindowApply
datasets={}
resamplers= {'Random under-sampling': RandomUnderSampler(),
             'Random over-sampling': RandomOverSampler(),
             'Tomek links under-sampling': TomekLinks(sampling_strategy='majority'),
             'Cluster Centroids under-sampling': ClusterCentroids(),
             'SMOTE over-sampling': SMOTE(sampling_strategy='minority')}

for name, resampler in resamplers.items():
  X_retrain,y_retrain= resampler.fit_resample(X_train, y_train)
  datasets[name]=[X_retrain,y_retrain,X_test, y_test]
  print(name,X_retrain.shape,y_retrain.shape)

Random under-sampling (2160, 94) (2160,)
Random over-sampling (6342, 94) (6342,)
Tomek links under-sampling (4246, 94) (4246,)
Cluster Centroids under-sampling (2160, 94) (2160,)
SMOTE over-sampling (6342, 94) (6342,)
CPU times: user 4min 46s, sys: 2min 37s, total: 7min 24s
Wall time: 3min 55s


In [ ]:
from pandas.core.common import random_state
classifiers = {
    "LogisiticRegression": LogisticRegression(class_weight='balanced', random_state=SEED),
    "KNearest": KNeighborsClassifier(),
    # "Support Vector Classifier": SVC(class_weight='balanced', probability=True, random_state=SEED),
    "DecisionTreeClassifier": DecisionTreeClassifier(class_weight='balanced', random_state=SEED),
    "RandomForest": RandomForestClassifier(class_weight= 'balanced', random_state=SEED),
    "XGBClassifier": xgb.XGBRFClassifier(tree_method="hist", random_state=SEED)
}
scorer={'f1_scorer': make_scorer(f1_score),
        'auc_scorer': make_scorer(roc_auc_score),
        'precision': make_scorer(precision_score),
        'recall': make_scorer(recall_score)}

In [ ]:
%%time
models={}
recs=[]
for dk, v in datasets.items():
  for key, classifier in classifiers.items():
    print(dk, key)
    clear_output(wait= True)
    classifier.fit(v[0], v[1])
    training_score = cross_validate(classifier,v[0], v[1], cv=5, scoring= scorer)
    y_hat= classifier.predict(v[2])
    rec=[dk+': '+key, dk, key]
    rec.extend(list(np.array(list(training_score.values())).mean(axis=1)[2:]))
    # # roc_auc_score(v[3], classifier.predict_proba(v[2])[:, 1])
    rec.extend([f1_score(v[3], y_hat),roc_auc_score(v[3], classifier.predict_proba(v[2])[:, 1]), precision_score(v[3], y_hat), recall_score(y_hat, v[3])])
    recs.append(rec)
    # models[dk+': '+key]= [classifier, v[0], v[1], v[2], v[4], v[5]]

SMOTE over-sampling XGBClassifier


In [ ]:
results= pd.DataFrame.from_records(recs)
results.columns=['DataClassifier', 'Sampling Strategy', 'Classifiers', 'train_f1', 'train_AUC', 'train_precision', 'train_recall','test_f1', 'test_AUC', 'test_precision', 'test_recall']

In [ ]:
results.loc[results.test_AUC> .52]

,DataClassifier,Sampling Strategy,Classifiers,train_f1,train_AUC,train_precision,train_recall,test_f1,test_AUC,test_precision,test_recall
4,Random under-sampling: RandomForest,Random under-sampling,RandomForest,0.504182,0.491667,0.490838,0.520370,0.353029,0.523859,0.274419,0.274419
6,Random over-sampling: LogisiticRegression,Random over-sampling,LogisiticRegression,0.587138,0.554088,0.546798,0.635772,0.371839,0.522351,0.275862,0.275862
10,Random over-sampling: RandomForest,Random over-sampling,RandomForest,0.632075,0.614635,0.604402,0.663522,0.350081,0.523298,0.285337,0.285337
11,Random over-sampling: XGBClassifier,Random over-sampling,XGBClassifier,0.402983,0.533112,0.554497,0.327037,0.270642,0.521184,0.298734,0.298734
12,Tomek links under-sampling: LogisiticRegression,Tomek links under-sampling,LogisiticRegression,0.368165,0.521626,0.269627,0.582407,0.374502,0.521553,0.274052,0.274052
17,Tomek links under-sampling: XGBClassifier,Tomek links under-sampling,XGBClassifier,0.000000,0.500000,0.000000,0.000000,0.000000,0.522422,0.000000,0.000000


## Base Column NULLS

In [ ]:
for i in [.05,.1,.2,.3,.4,.5,.6,.65, .7,.75, .8,.9,.95,.99]:
  flg= bf1.isnull().sum().sort_values()/ len(bf1) < i
  print(round(1-i,2), len(bf1.isnull().sum()[flg]))

0.95 0
0.9 1
0.8 1
0.7 1
0.6 2
0.5 2
0.4 2
0.35 2
0.3 2
0.25 3
0.2 6
0.1 93
0.05 263
0.01 327


In [ ]:
flg= bf1.isnull().sum().sort_values()/ len(bf1) < .8
bf1.isnull().sum()[flg]

COMPUTERS_INTERNET-VISIT-YOUTUBE_COM_IN_THE_LAST_30_DAYS                          4746
PERSONA_HOME-HOUSEHOLD_COMPOSITION_PRESENCE_OF_CHILDREN_PRESENCE_OF_CHILD_0-18    4483
SHOPPING_CLOTHING-I_LIKE_TO-EXPERIMENT_WITH_NEW_STYLES                            4764
SHOPPING_CLOTHING-REALLY_ENJOY_CLOTHES_SHOPPING                                   4754
gender                                                                             595
birth_year                                                                        1874
dtype: int64

In [ ]:
bf2= bf1.loc[:,bf1.isnull().sum()[flg].index.tolist()]

## Aggretated to 0/1

In [ ]:
mf= pd.read_excel('/content/BDEX New Aggregated Columns.xlsx')
mf.drop(index=[0,1,2], axis=0, inplace=True)
mf.NewAggregateSet1.fillna(mf.NewAggregateSet2, axis=0, inplace= True)
mapr= mf[['BDEX May 2022 Columns','NewAggregateSet1']].set_index('BDEX May 2022 Columns').to_dict()
mapr= mapr['NewAggregateSet1']

In [ ]:
bf3= bf1.melt(ignore_index=False)

In [ ]:
bf3['NewVar']= bf3.variable.map(mapr)

In [ ]:
bf3.loc[bf3.variable.isin(['PERSONA_EDUCATION-ASSOCIATE_DEGREE',
       "PERSONA_EDUCATION-BACHELOR'S_DEGREE",
       'PERSONA_EDUCATION-GRADUATE_DEGREE']), 'NewVar']= 'CollegeEducated'

In [ ]:
bf3.loc[bf3.variable.isin(['PERSONA_EDUCATION-HIGH_SCHOOL_DIPLOMA',
       'PERSONA_EDUCATION-LESS_THAN_12TH_GRADE',
       'PERSONA_EDUCATION-LESS_THAN_9TH_GRADE',
       'PERSONA_EDUCATION-SOME_COLLEGE']), 'NewVar']= 'HSorLess'

SyntaxError: ignored

In [ ]:
bf3.head()

,variable,value,NewVar
email_address,,,
MAZ5695@GMAIL.COM,COMPUTERS_INTERNET-AMAZON_SITES_LAST_3MONTH,NaN,OnlineShopping
SELISELAM5@GMAIL.COM,COMPUTERS_INTERNET-AMAZON_SITES_LAST_3MONTH,NaN,OnlineShopping
SATANSSONTHEDEVIL.BJ@GMAIL.COM,COMPUTERS_INTERNET-AMAZON_SITES_LAST_3MONTH,NaN,OnlineShopping
RUTKAMAKAYLA@GMAIL.COM,COMPUTERS_INTERNET-AMAZON_SITES_LAST_3MONTH,NaN,OnlineShopping
DAVIS.CHRIS1A@GMAIL.COM,COMPUTERS_INTERNET-AMAZON_SITES_LAST_3MONTH,NaN,OnlineShopping


In [ ]:
bf3.loc[bf3.NewVar== 'Gender', 'value']= bf3.loc[bf3.NewVar== 'Gender', 'value'].map({'M': 0, 'F':1})

In [ ]:
bf4= pd.crosstab(index= bf3.index, columns=bf3.NewVar, values= bf3.value.fillna(0), aggfunc='max')

In [ ]:
bf4['Age']= 2022- bf4.Age.replace(0.0,np.nan).fillna(bf3.loc[bf3.NewVar== 'Age', 'value'].median())

In [ ]:
mapdict={}
for c in bf4.columns:
  print(c, sum(bf4[c]==1)/len(bf4))
  mapdict[c]= sum(bf4[c]==1)/len(bf4)

Age 0.0
Blue Collar - Casual Salaried Worker 0.562983698336901
COMPUTERS_MOBILE_COMPUTING 0.24897085460233823
CareerInterest 0.34101761896920796
Clothing - UnWashable 0.46352708710686646
Clothing - Washable 0.722048410999506
CollegeEducated 0.030792030298040507
CurrentAffairs 0.701959492837148
Gender 0.5163839947307756
HSorLess 0.10620780503869587
LAUNDRY_DETERGENT 0.5555738514737362
Online Travel 0.128931335419068
OnlineShopping 0.8164004610571381
OnlineSportsEntertainment 0.6706734727482299
PERSONA_FINANCES_1 0.02601679565288984
PERSONA_FINANCES_2 0.017618969207969702
PERSONA_FINANCES_3 0.05351556067841265
PERSONA_FINANCES_4 0.010538448872056645
PERSONA_FINANCES_5 0.007903836654042483
PERSONA_FINANCES_6 0.01136176519018607
PERSONA_FINANCES_7 0.012185081508315494
PERSONA_HOME 0.26181458916515726
Real Estate & Finance 0.618969207969702
SHOPPING_CHILDREN 0.14292771282726824
Self-Employed 0.6013502387617322
Social Media & Networking 0.9074592458422526
WellnessHealth 0.30298040507162854
W

In [ ]:
feat1= pd.Series(mapdict)[pd.Series(mapdict)> .4].index.tolist()